In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder


In [12]:
data = pd.read_excel('/content/Training data.xlsx')
x_train = np.array(data.iloc[:,0:8])
y_train = np.array(data.iloc[:,8]).reshape(-1,1)

In [13]:
def feature_changing(x_train):
    le = LabelEncoder()
    x_train[:, 0] = le.fit_transform(x_train[:, 0])
    x_train[:, 1] = le.fit_transform(x_train[:, 1])
    return x_train

x_train = feature_changing(x_train)

In [14]:
def z_score(x_train):
    x_mean = np.mean(x_train, axis=0)
    x_std = np.std(x_train, axis=0)
    x_train = (x_train - x_mean) / x_std
    return x_train, x_std, x_mean

In [16]:
def cost(x_train, y_train, w, b):
    m = len(y_train) # Define m here as the length of y_train
    predictions = np.dot(x_train, w) + b
    loss = np.sum((predictions - y_train) ** 2) / (2 * m)
    return loss

In [17]:
def gradient_descent(x_train, y_train, w, b, learning_rate=0.01, epochs=1000):
    m = len(y_train)
    for i in range(epochs):
        predictions = np.dot(x_train, w) + b
        error = predictions - y_train
        w_gradient = np.dot(x_train.T, error) / m
        b_gradient = np.sum(error) / m
        w -= learning_rate * w_gradient
        b -= learning_rate * b_gradient
    return w, b

In [18]:
x_train = x_train.astype(np.float64)
x_train, x_std, x_mean = z_score(x_train)

np.random.seed(2147483647)
w = np.random.randn(x_train.shape[1],1)
b = np.random.randn(1)

old_cost = 0
current_cost = cost(x_train, y_train, w, b)

while abs(old_cost - current_cost) > 0.00001:
    old_cost = current_cost
    w, b = gradient_descent(x_train, y_train, w, b)
    current_cost = cost(x_train, y_train, w, b)

x_predict = pd.read_excel('/content/Test data.xlsx').iloc[:,:8].to_numpy()
x_predict = feature_changing(x_predict)
x_predict = (x_predict - x_mean) / x_std
ans = pd.read_excel('/content/Test data.xlsx').iloc[:,8].to_numpy()

y_predict = np.dot(x_predict, w) + b

accuracy = 0
for dim in range(len(ans)):
    if abs(y_predict[dim] - ans[dim]) < 0.5:  # do not change the tolerance as you'll be checked on +- 0.5 error only
        accuracy += 1
accuracy = round(accuracy * 100 / len(ans), 2)
ok = 'Congratulations' if accuracy > 95 else 'Optimization required'
print(f"{ok}, your accuracy is {accuracy}%")

Congratulations, your accuracy is 100.0%
